In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [21]:
train = pd.read_csv('train.csv', index_col='user_id')
test = pd.read_csv('test.csv', index_col='user_id')
pd.set_option('display.max_rows', 10)

train_x = train.drop(['Attrition', 'EmployeeNumber', 'EmployeeCount', 'StandardHours'], axis=1)
train_y = train['Attrition']
test.drop(['EmployeeNumber', 'EmployeeCount', 'StandardHours'], axis=1, inplace=True)

In [22]:
# One-Hot Encoder
object_train_x = [col for col in train_x.columns if train_x[col].dtype == 'object']
numerical_train_x = [col for col in train_x.columns if train_x[col].dtype != 'object']
object_test = [col for col in test.columns if test[col].dtype == 'object']
numerical_test = [col for col in test.columns if test[col].dtype != 'object']

train_x_object = train_x[object_train_x]
train_x_numerical = train_x[numerical_train_x]
test_object = test[object_test]
test_numerical = test[numerical_test]

oh = OneHotEncoder(sparse=False)
train_x_object_onehot = pd.DataFrame(oh.fit_transform(train_x_object))
test_object_onehot = pd.DataFrame(oh.fit_transform(test_object))

train_x_object_onehot.index = train_x.index
test_object_onehot.index = test.index

# Min-Max Scaler
#mms = MinMaxScaler()
#train_x_numerical = pd.DataFrame(mms.fit_transform(train_x_numerical))
#train_x_numerical.index = train_x.index

oh_train_x = pd.concat([train_x_numerical, train_x_object_onehot], axis=1)
oh_test = pd.concat([test_numerical, test_object_onehot], axis=1)

# Label Encoder
lenc = LabelEncoder()
train_y = pd.DataFrame(lenc.fit_transform(train_y))
train_y.index = train_x.index

In [23]:
# Train-Test Split
train_x_train, train_x_test, train_y_train, train_y_test = train_test_split(oh_train_x, train_y, train_size=0.8, test_size=0.2, random_state=23)

In [24]:
lr = LogisticRegression(solver='liblinear', multi_class='auto')
lr.fit(train_x_train, train_y_train)
results = lr.predict_proba(oh_test)
lr.score(train_x_test, train_y_test)

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.902542372881356

In [18]:
test_y = test.drop(test.columns[0:34], axis=1, inplace=False)
test_y.insert(0, 'Attrition', results[:, 1])
test_y = test_y.reset_index()
test_y.to_csv('./output.csv', index=False)

In [25]:
# 得到准确率(基于训练集)
acc = round(lr.score(train_x_test, train_y_test), 6)
print(u'score准确率为 %.4lf' % acc)
# 使用K折交叉验证 统计准确率
print(u'cross_val_score准确率为 %.4lf' % np.mean(cross_val_score(lr, train_x_test, train_y_test, cv=10)))

score准确率为 0.9025


D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

cross_val_score准确率为 0.8607
